# Driveway Footprint Detection 
Notebook 1

In [81]:
import os
os.environ['OPENCV_IO_ENABLE_JASPER']='True' # reading jp2000 images
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
import rasterio as rio
from rasterio.windows import Window
import random
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torchvision.models as models

## Data exploration

In [119]:
city = 'kw_3band'
crop_size = 512
EXPERIMENT_NAME = 'test1'
TOTAL_EPOCHS = 100
LR = 1e-4
WD = 5e-4
test_freq = 5

PATH_DIRECTORY = '/mnt/mount-point-directory/building_footprint/'
PATH_DATA = os.path.join(PATH_DIRECTORY, 'data')

PATH_DATA_BUILDING_POLYGON = os.path.join(PATH_DIRECTORY, 'data', 'building_polygon', 'tiles_jp2')
PATH_DATA_ROAD_FILL = os.path.join(PATH_DIRECTORY, 'data', 'road_fill', 'tiles_jp2')
PATH_DATA_CITY = os.path.join('/mnt/mount-point-directory/datasets/', city, 'VRT')

PATH_DATA_DRIVEWAY_POLYGON = os.path.join(PATH_DIRECTORY, 'data', 'driveways_polygon', 'tiles_jp2')


In [3]:
ls '/mnt/mount-point-directory/building_footprint/data'

building_polygon/  driveways_polygon/  road_fill/  road_polyline/


In [4]:
list_building_polygon = os.listdir('/mnt/mount-point-directory/building_footprint/data/building_polygon/tiles_jp2/')

In [5]:
list_images = os.listdir(os.path.join(PATH_DATA_CITY))
list_road = os.listdir(os.path.join(PATH_DATA_ROAD_FILL))
list_building = os.listdir(os.path.join(PATH_DATA_BUILDING_POLYGON))

list_driveway = os.listdir(os.path.join(PATH_DATA_DRIVEWAY_POLYGON))

In [6]:
# for name in list_images:
#     print(name)
#     new_name = name.split('_')[-1]
#     os.rename(os.path.join(PATH_DATA_CITY, name), os.path.join(PATH_DATA_CITY, new_name))

In [7]:
names_final = list((set.intersection(set(list_images),set(list_road))))
file_name = names_final[5]

### Data split
Train 70%
Validation 15%
Test 15%

In [8]:
train_nos = int(len(names_final) * .7)
val_nos = int(len(names_final) * .15)
test_nos = len(names_final) - train_nos - val_nos

In [9]:
with open(os.path.join(PATH_DIRECTORY, 'train.txt'), 'w') as f:
    for item in names_final[:train_nos]:
        f.write("%s\n" % item)
        
with open(os.path.join(PATH_DIRECTORY, 'val.txt'), 'w') as f:
    for item in names_final[train_nos : train_nos + val_nos]:
        f.write("%s\n" % item)
        
with open(os.path.join(PATH_DIRECTORY, 'test.txt'), 'w') as f:
    for item in names_final[:test_nos]:
        f.write("%s\n" % item)

### Dataloaders

In [10]:
PATH_DIRECTORY = '/mnt/mount-point-directory/building_footprint/'
PATH_DATA = os.path.join(PATH_DIRECTORY, 'data')

PATH_DATA_BUILDING_POLYGON = os.path.join(PATH_DIRECTORY, 'data', 'building_polygon', 'tiles_jp2')
PATH_DATA_ROAD_FILL = os.path.join(PATH_DIRECTORY, 'data', 'road_fill', 'tiles_jp2')
PATH_DATA_CITY = os.path.join('/mnt/mount-point-directory/datasets/', city, 'VRT')

PATH_DATA_DRIVEWAY_POLYGON = os.path.join(PATH_DIRECTORY, 'data', 'driveways_polygon', 'tiles_jp2')

In [11]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as TF
from torchvision import transforms
import csv
import PIL
from tqdm import tqdm

In [170]:
class TrainDrivewayDataset(Dataset):
    def __init__(self, path_directory, mode):
        self.path_directory = path_directory
        
        self.namefile = open(os.path.join(self.path_directory, 'train.txt'), 'r')
        self.reader = csv.reader(self.namefile)
        self.image_names = [row[0] for row in self.reader]
        
        self.crop_size = 768
        self.reshape = True
        self.reshape_size = 512
        
        self.to_pil = transforms.ToPILImage()
        self.color_jit = transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.3, hue=0.05)
        self.to_tensor = transforms.ToTensor()
        
    def __len__(self):
#         return(len(self.image_names))
        return(2)
        
    def __getitem__(self, index):
        # randomly choose an image
        file_name = random.choice(self.image_names)
        
        # prepapre reading paths
        path_img = os.path.join('/mnt/mount-point-directory/datasets/', city, 'VRT')
        path_building = os.path.join(self.path_directory, 'data', 'building_polygon', 'tiles_jp2')
        path_road = os.path.join(self.path_directory, 'data', 'road_fill', 'tiles_jp2')
        path_driveway = os.path.join(self.path_directory, 'data', 'driveways_polygon', 'tiles_jp2')
        
        # read src datasets
        src_img = rio.open(os.path.join(path_img, file_name), mode = 'r')
        src_building = rio.open(os.path.join(path_building, file_name), mode ='r')
        src_road = rio.open(os.path.join(path_road, file_name), mode = 'r')
        src_driveway = rio.open(os.path.join(path_driveway, file_name), mode = 'r')
        
        meta = src_img.meta
        x_lim = meta['width'] - self.crop_size 
        y_lim = meta['height'] - self.crop_size
        
        # find a random path until there is some data in it
#         while True: 
#             col_off = np.random.randint(0, x_lim)
#             row_off = np.random.randint(0, y_lim)

#             window = Window(col_off=col_off,
#                 row_off=row_off,
#                 width=self.crop_size,
#                 height=self.crop_size)

#             _mask = src_driveway.read(window = window)
#             print(window)
#             if _mask.any():
#                 break

        col_off = np.random.randint(0, x_lim)
        row_off = np.random.randint(0, y_lim)

        window = Window(col_off=col_off,
            row_off=row_off,
            width=self.crop_size,
            height=self.crop_size)

        _mask = src_driveway.read(window = window)
        
                
        # read images, if you get nullpointer error in img read, there is a channel issue. Reformat the images.
        _img = src_img.read((1,2,3), window = window)
        _mask_building = src_building.read(1, window = window)
        _mask_road = src_road.read(1, window = window)
        _mask_driveway = src_driveway.read(1, window = window)
        
        _input_tensor, _mask = self.transform(_img, _mask_building, _mask_road, _mask_driveway)
        
        return _input_tensor, _mask
        
    def transform(self, img, mask_building, mask_road, mask_driveway):
        """
        Input Tensor, Ouput Tensor
        
        """
        
        # To Tensor
        img = torch.tensor(img)
        mask_building = torch.tensor(mask_building)
        mask_road = torch.tensor(mask_road)
        mask_driveway = torch.tensor(mask_driveway)
        
        # To PIL image
        image = self.to_pil(img)
        mask_building = self.to_pil(mask_building)
        mask_road = self.to_pil(mask_road)
        mask_driveway = self.to_pil(mask_driveway)
        
        # Resize
        image = TF.resize(image, size = self.reshape_size, interpolation=PIL.Image.NEAREST)
        mask_building = TF.resize(mask_building, size = self.reshape_size, interpolation=PIL.Image.NEAREST)
        mask_road = TF.resize(mask_road, size = self.reshape_size, interpolation=PIL.Image.NEAREST)
        mask_driveway = TF.resize(mask_driveway, size = self.reshape_size, interpolation=PIL.Image.NEAREST)

        # Random horizontal flip
        if random.random() > 0.5:
            image = TF.hflip(image)
            mask_building= TF.hflip(mask_building)
            mask_road= TF.hflip(mask_road)
            mask_driveway= TF.hflip(mask_driveway)
            
        # Random Vertical flip
        if random.random() > 0.5:
            image = TF.vflip(image)
            mask_building = TF.vflip(mask_building)
            mask_road = TF.vflip(mask_road)
            mask_driveway = TF.vflip(mask_driveway)
        
        # Color Jitter Image
        image = self.color_jit(image)
        
        # Change to tensors
        image = self.to_tensor(image)
        mask_building = self.to_tensor(mask_building)
        mask_road = self.to_tensor(mask_road)
        mask_driveway = self.to_tensor(mask_driveway)
        
        # Merge input tensors to 5 channel, 3 image 1 building 1 road
        _input_stacked = torch.cat((image, mask_building, mask_road))
        
        return _input_stacked, mask_driveway

In [171]:
class ValTestDrivewayDataset(Dataset):
    def __init__(self, path_directory, mode):
        self.path_directory = path_directory
        
        self.namefile = open(os.path.join(self.path_directory, (mode + '.txt')), 'r')
        self.reader = csv.reader(self.namefile)
        self.image_names = [row[0] for row in self.reader]
        
        self.crop_size = 768
        self.reshape = True
        self.reshape_size = 512
        
        self.to_pil = transforms.ToPILImage()
#         self.color_jit = transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.3, hue=0.05)
        self.to_tensor = transforms.ToTensor()
        
    def __len__(self):
#         return(len(self.image_names))
        return(5)
        
    def __getitem__(self, index):
        # randomly choose an image
        file_name = random.choice(self.image_names)
        
        # prepapre reading paths
        path_img = os.path.join('/mnt/mount-point-directory/datasets/', city, 'VRT')
        path_building = os.path.join(self.path_directory, 'data', 'building_polygon', 'tiles_jp2')
        path_road = os.path.join(self.path_directory, 'data', 'road_fill', 'tiles_jp2')
        path_driveway = os.path.join(self.path_directory, 'data', 'driveways_polygon', 'tiles_jp2')
        
        # read src datasets
        src_img = rio.open(os.path.join(path_img, file_name), mode = 'r')
        src_building = rio.open(os.path.join(path_building, file_name), mode ='r')
        src_road = rio.open(os.path.join(path_road, file_name), mode = 'r')
        src_driveway = rio.open(os.path.join(path_driveway, file_name), mode = 'r')
        
        meta = src_img.meta
        x_lim = meta['width'] - self.crop_size 
        y_lim = meta['height'] - self.crop_size
        
        # find a random path until there is some data in it
#         while True: 
#             col_off = np.random.randint(0, x_lim)
#             row_off = np.random.randint(0, y_lim)

#             window = Window(col_off=col_off,
#                 row_off=row_off,
#                 width=self.crop_size,
#                 height=self.crop_size)

#             _mask = src_driveway.read(window = window)
#             print(window)
#             if _mask.any():
#                 break

        col_off = np.random.randint(0, x_lim)
        row_off = np.random.randint(0, y_lim)

        window = Window(col_off=col_off,
            row_off=row_off,
            width=self.crop_size,
            height=self.crop_size)

        _mask = src_driveway.read(window = window)
        
                
        # read images, if you get nullpointer error in img read, there is a channel issue. Reformat the images.
        _img = src_img.read((1,2,3), window = window)
        _mask_building = src_building.read(1, window = window)
        _mask_road = src_road.read(1, window = window)
        _mask_driveway = src_driveway.read(1, window = window)
        
        _input_tensor, _mask = self.transform(_img, _mask_building, _mask_road, _mask_driveway)
        
        return _input_tensor, _mask
        
    def transform(self, img, mask_building, mask_road, mask_driveway):
        """
        Input Tensor, Ouput Tensor
        
        """
        
        # To Tensor
        img = torch.tensor(img)
        mask_building = torch.tensor(mask_building)
        mask_road = torch.tensor(mask_road)
        mask_driveway = torch.tensor(mask_driveway)
        
        # To PIL image
        image = self.to_pil(img)
        mask_building = self.to_pil(mask_building)
        mask_road = self.to_pil(mask_road)
        mask_driveway = self.to_pil(mask_driveway)
        
        # Resize
        image = TF.resize(image, size = self.reshape_size, interpolation=PIL.Image.NEAREST)
        mask_building = TF.resize(mask_building, size = self.reshape_size, interpolation=PIL.Image.NEAREST)
        mask_road = TF.resize(mask_road, size = self.reshape_size, interpolation=PIL.Image.NEAREST)
        mask_driveway = TF.resize(mask_driveway, size = self.reshape_size, interpolation=PIL.Image.NEAREST)
        
        # Change to tensors
        image = self.to_tensor(image)
        mask_building = self.to_tensor(mask_building)
        mask_road = self.to_tensor(mask_road)
        mask_driveway = self.to_tensor(mask_driveway)
        
        # Merge input tensors to 5 channel, 3 image 1 building 1 road
        _input_stacked = torch.cat((image, mask_building, mask_road))
        
        return _input_stacked, mask_driveway

### Prepare DeepLabV3 segmentation model

In [172]:
import torchvision
import torchvision.models as models

In [173]:
model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=False, num_classes=1, aux_loss=None)

In [174]:
model.backbone.conv1 = nn.Conv2d(5, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

### Initilize dataloaders

In [175]:
train_ds = TrainDrivewayDataset(PATH_DIRECTORY, mode = 'train')
train_loader = DataLoader(train_ds, batch_size = 2, shuffle = True, num_workers = 16)

val_ds = ValTestDrivewayDataset(PATH_DIRECTORY, mode = 'val')
val_loader = DataLoader(val_ds, batch_size = 4, shuffle = True, num_workers = 16)

### Imports

In [176]:
from utils import loss, util
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn.functional as F

### Loss

In [177]:
def dice_loss(y_true, y_pred, smooth = 1e-6):
    pred = y_pred.contiguous()
    target = y_true.contiguous()

    intersection = (pred * target).sum(dim=2).sum(dim=2)

    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))

    return loss.mean()

def calc_loss(pred, target, weight_bce):
    pred = pred
    bce = F.binary_cross_entropy_with_logits(pred, target)

    pred = torch.sigmoid(pred)
    dice = dice_loss(y_true = target, y_pred = pred)

    loss = bce * weight_bce + dice * (1-weight_bce)

    return loss

def compute_miou(y_pred, y_true):
    y_pred = y_pred.flatten()
    y_true = y_true.flatten()
    
    current = confusion_matrix(y_true, y_pred, labels=[0, 1])
    
    intersection = np.diag(current)
    ground_truth_set = current.sum(axis=1)
    predicted_set = current.sum(axis=0)
    union = ground_truth_set + predicted_set - intersection
    IoU = intersection / union.astype(np.float32)
    
    return np.mean(IoU)

### Logging

In [178]:
experiment_dir = os.path.join('./save/', EXPERIMENT_NAME)
util.ensure_dir(experiment_dir)

In [179]:
### Logging Files
train_file = "{}/{}_train_loss.txt".format(experiment_dir, city)
test_file = "{}/{}_test_loss.txt".format(experiment_dir, city)

train_loss_file = open(train_file, "w")
val_loss_file = open(test_file, "w")

In [180]:
num_gpus = torch.cuda.device_count()

In [181]:
if num_gpus > 1:
    print("Training with multiple GPUs ({})".format(num_gpus))
    model = nn.DataParallel(model).cuda()
elif num_gpus == 1:
    print("Single Cuda Node is avaiable")
    model.cuda()
else:
    print("Training on CPU, GPU not available")

Training on CPU, GPU not available


## DO CHECK HERE

In [182]:
best_miou = 0
best_loss = 1e10
epochs = 1
total_epochs = TOTAL_EPOCHS

In [183]:
optimizer = torch.optim.Adam(
    model.parameters(), lr=LR, weight_decay=WD
)

In [184]:
def train():    
    model.train()
    optimizer.zero_grad()
    
    for i, data in enumerate(train_loader):
        _input, gt = sample
        _output = model(_input)
        
        loss = calc_loss(_output['out'], gt, weight_bce=0.5)
        
        loss.backward()
        optimizer.step()
        
        pred = (torch.sigmoid(_output['out']) > 0.5).numpy().astype(int)
        
        running_mIOU = compute_miou(y_pred=pred, y_true=gt)
        
        print('Train Epoch:{} --- Running Loss:{} --- Running mIOU:{}'.format(start_epoch, loss.item(), running_mIOU))

In [185]:
def save_checkpoint(loss, model, optimizer, experiment_dir):

    if torch.cuda.device_count() > 1:
        arch = type(model.module).__name__
    else:
        arch = type(model).__name__
    state = {
        "arch": arch,
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "loss": loss,
    }
    filename = os.path.join(
        experiment_dir, "checkpoint-loss-{:.4f}.pth.tar".format(loss)
    )
    torch.save(state, filename)
    os.rename(filename, os.path.join(experiment_dir, "model_best.pth.tar"))
    print("Saving current best: {} ...".format("model_best.pth.tar"))

In [186]:
def val():
    with torch.no_grad():
        val_loss = 0
        
        model.eval()

        for i, data in enumerate(val_loader):
            _input, gt = sample
            _output = model(_input)

            loss = calc_loss(_output['out'], gt, weight_bce=0.5)

            pred = (torch.sigmoid(_output['out']) > 0.5).numpy().astype(int)

            running_mIOU = compute_miou(y_pred=pred, y_true=gt)

            print('Running Loss:{} --- Running mIOU:{}'.format(loss.item(), running_mIOU))
            
            val_loss = val_loss + loss
        
        return val_loss

In [ ]:
while epochs < total_epochs:
    train()
    
    if epochs%test_freq == 0:
        val_loss = val()
        
        if val_loss < best_loss:
            best_loss = val_loss
            save_checkpoint(loss = best_loss, 
                            model = model, 
                            optimizer = optimizer, 
                            experiment_dir = experiment_dir)
    epochs = epochs + 1

Train Epoch:0 --- Running Loss:0.8070863485336304 --- Running mIOU:0.36496350868975574
Train Epoch:0 --- Running Loss:0.7916972637176514 --- Running mIOU:0.4571526663630442
Train Epoch:0 --- Running Loss:0.7745586633682251 --- Running mIOU:0.5170357903506669
Train Epoch:0 --- Running Loss:0.755042552947998 --- Running mIOU:0.5585140834896927
Train Epoch:0 --- Running Loss:0.736782431602478 --- Running mIOU:0.5989812038939125
Running Loss:0.8155452013015747 --- Running mIOU:0.5455327019887042
Running Loss:0.8155452013015747 --- Running mIOU:0.5455327019887042
Saving current best: model_best.pth.tar ...


In [8]:
src_img = rio.open(os.path.join(PATH_DATA_CITY, file_name), mode = 'r')
src_building = rio.open(os.path.join(PATH_DATA_BUILDING_POLYGON, file_name), mode ='r')
src_road = rio.open(os.path.join(PATH_DATA_ROAD_FILL, file_name), mode = 'r')
src_driveway = rio.open(os.path.join(PATH_DATA_DRIVEWAY_POLYGON, file_name), mode = 'r')

In [9]:
meta = src_img.meta
x_lim = meta['width'] - crop_size 
y_lim = meta['height'] - crop_size

In [10]:
while True: 
    col_off = np.random.randint(0, x_lim)
    row_off = np.random.randint(0, y_lim)

    window = Window(col_off=col_off,
        row_off=row_off,
        width=crop_size,
        height=crop_size)

    _mask = src_driveway.read(window = window)

    if _mask.any():
        break

In [11]:
_img = src_img.read((1,2,3), window = window)
_mask_building = src_building.read(1, window = window)
_mask_road = src_road.read(1, window = window)
_mask_driveway = src_driveway.read(1, window = window)

In [17]:
config = {
    'train':{
        'hello':1
    }
}

In [18]:
config

{'train': {'hello': 1}}

In [12]:
from torch.utils.data import Dataset
import torchvision.transforms.functional as TF
from torchvision import transforms

In [19]:
class TrainDrivewayDataset(Dataset):
    def __init__(self, image_names):
        
        
    def __len__(self):
        
    def __getitem__(self):
        

IndentationError: expected an indented block (<ipython-input-19-f87fcebfa447>, line 5)

In [ ]:
make train validation test set
